# Transforming

This notebook is used to experiment with and create new features for the DiaRisk project. The goal is to engineer clinically relevant features that improve the predictive power of the model.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define file path for the raw data
RAW_DATA_PATH = "/Users/kartavyasoni/Documents/Projects/Data Science/DiaRisk---Readmission-Prediction/Data/Raw/diabetic_data.csv"

# Load the raw dataset
df = pd.read_csv(RAW_DATA_PATH)

# Check the shape of the dataset
print("\nDataset shape:", df.shape)

# Display the first few rows
df.head()


Dataset shape: (101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


## Data Cleaning

In [2]:
# Dataset shape
print("\nDataset shape:", df.shape)

# Columns with missing values
missing_values = df.isnull().sum()
print("\nMissing values in each column:\n", missing_values[missing_values > 0])

# Remove columns `max_glucose_serum` and `A1Cresult` as they have more than 50% missing values
df = df.drop(columns=['max_glu_serum', 'A1Cresult'])

print("\nDataset shape after removing columns with >50% missing values:", df.shape)


Dataset shape: (101766, 50)

Missing values in each column:
 max_glu_serum    96420
A1Cresult        84748
dtype: int64

Dataset shape after removing columns with >50% missing values: (101766, 48)


In [3]:
# Normalized value counts for categorical features in percentage
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nValue count for {col} (Normalized):")
    print(df[col].value_counts(normalize=True) * 100)


Value count for race (Normalized):
race
Caucasian          74.778413
AfricanAmerican    18.876639
?                   2.233555
Hispanic            2.001651
Other               1.479866
Asian               0.629876
Name: proportion, dtype: float64

Value count for gender (Normalized):
gender
Female             53.758623
Male               46.238429
Unknown/Invalid     0.002948
Name: proportion, dtype: float64

Value count for age (Normalized):
age
[70-80)     25.615628
[60-70)     22.092840
[50-60)     16.956547
[80-90)     16.898571
[40-50)      9.516931
[30-40)      3.709490
[90-100)     2.744532
[20-30)      1.628245
[10-20)      0.679009
[0-10)       0.158206
Name: proportion, dtype: float64

Value count for weight (Normalized):
weight
?            96.858479
[75-100)      1.312816
[50-75)       0.881434
[100-125)     0.614154
[125-150)     0.142484
[25-50)       0.095317
[0-25)        0.047167
[150-175)     0.034393
[175-200)     0.010809
>200          0.002948
Name: proportion, dt

In [4]:
# Remove columns that have very high missining values as placeholders and are not useful for analysis
columns_to_remove = ['weight', 'payer_code']
df = df.drop(columns=columns_to_remove, errors='ignore')

print("\nDataset shape after removing unnecessary columns:", df.shape)


Dataset shape after removing unnecessary columns: (101766, 46)


In [5]:
# Replace placeholders in 'medical_specialty' with 'Missing' for better clarity
df['medical_specialty'] = df['medical_specialty'].replace('?', 'Missing')

# Remove rows with placeholders indicating missing data.
placeholders = ['?', 'Unknown/Invalid']
df = df.replace(placeholders, np.nan).dropna()

print("\nDataset shape after removing rows with placeholders for missing data:", df.shape)


Dataset shape after removing rows with placeholders for missing data: (98052, 46)


In [6]:
# calculate unique values for patient_nbr
unique_patients = df['patient_nbr'].nunique()
print(f"\nNumber of unique patients: {unique_patients}")

# Keep only the first occurrence of each patient for bias reduction
df = df.drop_duplicates(subset='patient_nbr', keep='first')

print(f"\nDataset shape after keeping only the first occurrence of each patient: {df.shape}")


Number of unique patients: 68629

Dataset shape after keeping only the first occurrence of each patient: (68629, 46)


In [7]:
# Normalized value counts for categorical features in percentage
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nValue counts for {col} (normalized):")
    print(df[col].value_counts(normalize=True) * 100)


Value counts for race (normalized):
race
Caucasian          76.996605
AfricanAmerican    18.454298
Hispanic            2.152151
Other               1.690248
Asian               0.706698
Name: proportion, dtype: float64

Value counts for gender (normalized):
gender
Female    53.303997
Male      46.696003
Name: proportion, dtype: float64

Value counts for age (normalized):
age
[70-80)     25.706334
[60-70)     22.459893
[50-60)     17.483863
[80-90)     16.388116
[40-50)      9.514928
[30-40)      3.680660
[90-100)     2.682539
[20-30)      1.470224
[10-20)      0.520188
[0-10)       0.093255
Name: proportion, dtype: float64

Value counts for medical_specialty (normalized):
medical_specialty
Missing                   48.427050
InternalMedicine          15.130630
Family/GeneralPractice     7.134010
Emergency/Trauma           6.409827
Cardiology                 6.065949
                            ...    
SportsMedicine             0.001457
Dermatology                0.001457
Proctology  

## Data Typing (Casting)

In [8]:
# In column `race` keep 'Caucasian', 'AfricanAmerican' and replace others with 'Other'
df['race'] = df['race'].apply(lambda x: x if x in ['Caucasian', 'AfricanAmerican'] else 'Other')
print("\nValue counts for race after grouping others:\n", df['race'].value_counts(normalize=True) * 100)


Value counts for race after grouping others:
 race
Caucasian          76.996605
AfricanAmerican    18.454298
Other               4.549097
Name: proportion, dtype: float64


In [9]:
# In the `age` column, group ages into broader categories
age_bins = [0, 50, 70, 120]
age_labels = ['<50', '50-70', '>70']

# Extract the lower bound of the age range and convert it to integers
df['age'] = df['age'].str.extract(r'(\d+)-')[0].astype(int)

# Group ages into broader categories
df['age'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

print("\nValue counts for age after grouping:\n", df['age'].value_counts(normalize=True) * 100)


Value counts for age after grouping:
 age
>70      44.776989
50-70    39.943756
<50      15.279255
Name: proportion, dtype: float64


In [10]:
# In column `medical_specialty`, group less frequent specialties into 'Other'
specialty_counts = df['medical_specialty'].value_counts(normalize=True)
less_frequent_specialties = specialty_counts[specialty_counts < 0.05].index
df['medical_specialty'] = df['medical_specialty'].replace(less_frequent_specialties, 'Other')

print("\nValue counts for medical_specialty after grouping less frequent specialties:\n", df['medical_specialty'].value_counts(normalize=True) * 100)


Value counts for medical_specialty after grouping less frequent specialties:
 medical_specialty
Missing                   48.427050
Other                     16.832534
InternalMedicine          15.130630
Family/GeneralPractice     7.134010
Emergency/Trauma           6.409827
Cardiology                 6.065949
Name: proportion, dtype: float64


In [11]:
# Group diag_1, diag_2, diag_3 into broader categories based on ICD-9 chapters

# Define a dictionary for ICD-9 chapter ranges
icd9_chapters = {
    'Infectious and Parasitic Diseases': (1, 140),
    'Neoplasms': (140, 240),
    'Endocrine, Nutritional, and Metabolic Diseases': (240, 280),
    'Blood and Blood-Forming Organs': (280, 290),
    'Mental Disorders': (290, 320),
    'Nervous System': (320, 390),
    'Circulatory System': (390, 460),
    'Respiratory System': (460, 520),
    'Digestive System': (520, 580),
    'Genitourinary System': (580, 630),
    'Pregnancy, Childbirth, and the Puerperium': (630, 680),
    'Skin and Subcutaneous Tissue': (680, 710),
    'Musculoskeletal System': (710, 740),
    'Congenital Anomalies': (740, 760),
    'Perinatal Conditions': (760, 780),
    'Symptoms, Signs, and Ill-Defined Conditions': (780, 800),
    'Injury and Poisoning': (800, 1000)
}

# Define a function to map diagnosis codes to broader categories
def map_to_icd9_chapter(code):
    try:
        code = float(code)
        for chapter, (start, end) in icd9_chapters.items():
            if start <= code < end:
                return chapter
        return 'Other'
    except ValueError:
        return 'Other'

# Apply the mapping function to diag_1, diag_2, diag_3
diag_columns = ['diag_1', 'diag_2', 'diag_3']
for col in diag_columns:
    df[col] = df[col].apply(map_to_icd9_chapter)

# Display the value counts for the grouped categories
for col in diag_columns:
    print(f"\nValue counts for {col} after grouping:")
    print(df[col].value_counts(normalize=True) * 100)


Value counts for diag_1 after grouping:
diag_1
Circulatory System                                30.865960
Endocrine, Nutritional, and Metabolic Diseases    10.173542
Respiratory System                                 9.500357
Digestive System                                   8.999111
Symptoms, Signs, and Ill-Defined Conditions        7.799910
Injury and Poisoning                               6.648793
Musculoskeletal System                             5.665244
Genitourinary System                               4.942517
Neoplasms                                          3.842399
Infectious and Parasitic Diseases                  2.570342
Skin and Subcutaneous Tissue                       2.504772
Mental Disorders                                   2.124466
Other                                              1.312856
Nervous System                                     1.237086
Blood and Blood-Forming Organs                     0.939836
Pregnancy, Childbirth, and the Puerperium          0

In [ ]:
# Keep only the primary diagnosis for analysis and update column name
df['primiary_diagnosis'] = df['diag_1']
df = df.drop(columns=['diag_1','diag_2','diag_3'])

# Remove rows where primary diagnosis is 'Other'
df = df[df['primiary_diagnosis'] != 'Other']

print("\nDataset shape after dropping diag_2, diag_3 and removing 'Other' from primary diagnosis:", df.shape)


Dataset shape after dropping diag_2, diag_3 and removing 'Other' from primary diagnosis: (67728, 44)


In [13]:
# Remove column with only one value having a distribution higher than 98%
columns_to_check = df.columns
for col in columns_to_check:
    if df[col].value_counts(normalize=True).iloc[0] > 0.98:
        df = df.drop(columns=[col])

print("\nDataset shape after removing columns with high distribution:", df.shape)


Dataset shape after removing columns with high distribution: (67728, 28)


In [14]:
# Update all columns with values 'Up' and 'Down' into 'Changed'
for col in df.dtypes[df.dtypes == 'object'].index:
    df[col] = df[col].replace({'Up': 'Changed', 'Down': 'Changed'})

print("\nDataset shape after feature engineering:", df.shape)


Dataset shape after feature engineering: (67728, 28)
